# PSPNET Model
In this notebook I examine the performance of PSPNEt.

In [ ]:
import os
import sys
sys.path.insert(0, "..")
import gc
gc.enable()
os.environ['KERAS_BACKEND'] = 'tensorflow'
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import os
import sys
sys.path.insert(0, os.getcwd())
from utils.DataLoader import  DataLoader
from utils.utilities import *
from utils.visualization import *
from models.pspnet import PSPNet
import tensorflow as tf

### Loading the data and splitting them into training and validation sets

In [ ]:
tf.debugging.set_log_device_placement(True)
dl = DataLoader()
train_df, valid_df = dl.train_split(0.05, filterNan=True)
print("train set shape", train_df.shape)
print("validation set shape", valid_df.shape)

### The images are Loaded through a generator which is forwarded to a data augmentation generator

In [ ]:
grouped_images_gen = dl.get_grouped_images_gen(train_df, img_scaling=(3,3), f="adaptive_equalization") # applying filters increases dramatically the execution time
aug_gen = get_augmented_images_generator(grouped_images_gen)
valid_x, valid_y = next(dl.get_grouped_images_gen(valid_df, batch_size=900, img_scaling=(3,3), f="adaptive_equalization")) #valid_df.shape[0]))


## Initializing PSPNet Model

In [ ]:
t_x, t_y = next(aug_gen)
x_shape =np.squeeze(t_x[0]).shape
psp_model = PSPNet(x_shape, "../models/serialized/")
psp_model.compile()


## Training

In [ ]:
hl = psp_model.validate(aug_gen, train_df.shape[0], valid_set=(valid_x, valid_y), epochs=15, train_steps=400)
psp_model.show_loss(hl)


## Examine its Performance in Testing set.

In [ ]:
test_x, test_y = next(dl.get_grouped_images_gen(batch_size=40, train=False, f="adaptive_equalization"))
psp_model.examine_performance(test_x, test_y, load=True)